# Analyse de sentiments à l'aide d'un réseau  RNN

# Importation des fichiers et librairies

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install trax

     |████████████████████████████████| 522kB 8.3MB/s 
     |████████████████████████████████| 3.4MB 14.1MB/s 
     |████████████████████████████████| 215kB 37.5MB/s 
     |████████████████████████████████| 368kB 37.9MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 1.2MB 38.3MB/s 
     |████████████████████████████████| 1.9MB 43.2MB/s 
     |████████████████████████████████| 3.8MB 36.2MB/s 
     |████████████████████████████████| 3.2MB 43.1MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=4f12a4745e8a302aebecf7ad21fdbd06be6e2528ddfee0c7fb38d92bd6a42848
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import os
import nltk
import trax
from trax import layers as tl
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download("stopwords")


from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

from keras.models import Sequential

from keras.layers import Dense, Embedding, LSTM, Activation

import keras
import tensorflow as tf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Pré-traitement des données

In [4]:
data = pd.read_csv("gdrive/MyDrive/Projet_NLP/sample_data.csv",encoding = "ISO-8859-1",header = None)
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### 1-1- Sélection des données

In [136]:
data_to_begin = (data[799990:800090])   # On sélectionne un échantillon de données contenant des reviews positives et négatives
data_to_begin

,0,1,2,3,4,5
799990,0,2329204651,Thu Jun 25 10:28:26 PDT 2009,NO_QUERY,Badmantalking,"and I think it has a mind of its own,like it a..."
799991,0,2329204705,Thu Jun 25 10:28:27 PDT 2009,NO_QUERY,loveisnothing,@iaintnohomo Banana will be playing later. BU...
799992,0,2329204790,Thu Jun 25 10:28:27 PDT 2009,NO_QUERY,CJROSE218,@koolgirl37 read my tweet below
799993,0,2329204835,Thu Jun 25 10:28:27 PDT 2009,NO_QUERY,mattfca,My life http://mattf.ca/2009/06/24/yay-me/
799994,0,2329204987,Thu Jun 25 10:28:28 PDT 2009,NO_QUERY,360cookie,Tried to get the mutant Fawkes to follow me bu...
...,...,...,...,...,...,...
800085,4,1467825594,Mon Apr 06 22:23:38 PDT 2009,NO_QUERY,emmasaur28,eating ice-cream with chocolate topping
800086,4,1467825631,Mon Apr 06 22:23:38 PDT 2009,NO_QUERY,Rezdamc,@iamdiddy your telling me.. just finished doin...
800087,4,1467825641,Mon Apr 06 22:23:41 PDT 2009,NO_QUERY,laurengonewildx,@its_yvonne daaang i didn't know it was possib...
800088,4,1467825712,Mon Apr 06 22:23:42 PDT 2009,NO_QUERY,Andrael,@LaurUy West. Mostly San Jose and San Francisco.


In [137]:
reviews = []
fit_reviews = []
for i in data_to_begin.iloc(1)[0]:          # On créé une liste contenant le caractère positif (4) ou négatif (0)
  reviews.append(i)

for i in reviews:
  if i == 4:
    fit_reviews.append(1)                   # On modifie les 4 en 1 pour notre modèle
  else:
    fit_reviews.append(0)
print(fit_reviews)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### 1-2 Tokenisation, StopWords et Stemming

In [138]:
tweet_content = []                                                              
                                                                                
content = []
tokenizer = nltk.RegexpTokenizer(r'\w+')
english_stopwords = nltk.corpus.stopwords.words("english")

stemmer = LancasterStemmer()

for i in data_to_begin.iloc(1)[5]:
  tweet_content.append(tokenizer.tokenize(i.lower()))                           # On tokenize nos mots


tweet_content_without_stopwords = []
pos_or_neg_reviews = []

for i in tweet_content:
  tweet_sw = []
  for j in i:
    if j not in english_stopwords:
      tweet_sw.append(stemmer.stem(j))                                          # On retire les stopwords et on effectue le stemming
  tweet_content_without_stopwords.append([tweet_sw,data_to_begin.loc(1)[0]])


for j in range(len(tweet_content_without_stopwords)):
  content.append(tweet_content_without_stopwords[j][0])

print(content)                                                                  # content contient le contenu des tweets avec le pré-traitement effectué


[['think', 'mind', 'lik', 'alway', 'ring', 'tak', 'leak', 'jeez', 'mak', 'whol', 'thing', 'embarrass', 'mus'], ['iaintnohomo', 'banan', 'play', 'lat', 'shel', 'play', 'caro'], ['koolgirl37', 'read', 'tweet'], ['lif', 'http', 'mattf', 'ca', '2009', '06', '24', 'yay'], ['tri', 'get', 'mut', 'fawk', 'follow', 'lon', 'without', 'follow', 'nev', 'on'], ['sick', 'spend', 'day', 'lay', 'bed', 'list', 'taylorswift13'], ['gmail'], ['rest', 'peac', 'farrah', 'sad'], ['eric_urbane', 'sound', 'lik', 'riv', 'flag', 'ad', 'much', 'though'], ['resit', 'exam', 'sum', 'wish', 'work', 'hard', 'first', 'year', 'un'], ['lov', 'health4uandpets', 'u', 'guy', 'r', 'best'], ['im', 'meet', 'on', 'besty', 'tonight', 'cant', 'wait', 'girl', 'talk'], ['darealsunisakim', 'thank', 'twit', 'ad', 'sunis', 'got', 'meet', 'hin', 'show', 'dc', 'are', 'sweetheart'], ['sick', 'real', 'cheap', 'hurt', 'much', 'eat', 'real', 'food', 'plu', 'friend', 'mak', 'soup'], ['lovesbrooklyn2', 'effect', 'everyon'], ['productoffear', 

### 1-3 Définition du vocabulaire

In [139]:
# Building the vocabulary with the train set         (this might take a minute)
from collections import defaultdict

vocab = defaultdict(lambda: 0)
vocab['<PAD>'] = 1

for sentence in content:
  for word in sentence:
    if word not in vocab:
      vocab[word] = len(vocab) + 1
print('The length of the vocabulary is: ', len(vocab))                # Réutilisation de la fonction pour établir le vocabulaire
print(vocab)

The length of the vocabulary is:  587
defaultdict(<function <lambda> at 0x7f529c1c0290>, {'<PAD>': 1, 'think': 2, 'mind': 3, 'lik': 4, 'alway': 5, 'ring': 6, 'tak': 7, 'leak': 8, 'jeez': 9, 'mak': 10, 'whol': 11, 'thing': 12, 'embarrass': 13, 'mus': 14, 'iaintnohomo': 15, 'banan': 16, 'play': 17, 'lat': 18, 'shel': 19, 'caro': 20, 'koolgirl37': 21, 'read': 22, 'tweet': 23, 'lif': 24, 'http': 25, 'mattf': 26, 'ca': 27, '2009': 28, '06': 29, '24': 30, 'yay': 31, 'tri': 32, 'get': 33, 'mut': 34, 'fawk': 35, 'follow': 36, 'lon': 37, 'without': 38, 'nev': 39, 'on': 40, 'sick': 41, 'spend': 42, 'day': 43, 'lay': 44, 'bed': 45, 'list': 46, 'taylorswift13': 47, 'gmail': 48, 'rest': 49, 'peac': 50, 'farrah': 51, 'sad': 52, 'eric_urbane': 53, 'sound': 54, 'riv': 55, 'flag': 56, 'ad': 57, 'much': 58, 'though': 59, 'resit': 60, 'exam': 61, 'sum': 62, 'wish': 63, 'work': 64, 'hard': 65, 'first': 66, 'year': 67, 'un': 68, 'lov': 69, 'health4uandpets': 70, 'u': 71, 'guy': 72, 'r': 73, 'best': 74, 'im

In [140]:
maxlen = 0    
for word in content:
 if (len(word)) > maxlen:
   maxlen = len(word)

print(maxlen)                         # On détermine la longueur du mot le plus grand parmi notre échantillon de mot

18


###1-4 Mise en place du padding

In [141]:
padding = []
for word in content:
   L = []
   for i in range(maxlen):
     L.append(vocab['<PAD>'])
   for j in range (len(word)):
     L[j] = vocab[word[j]]                  # On transforme nos mots en vecteur de nombre pour les insérer dans notre modèle
   padding.append(L)

print(padding) 

[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1, 1, 1, 1, 1], [15, 16, 17, 18, 19, 17, 20, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [21, 22, 23, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [24, 25, 26, 27, 28, 29, 30, 31, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [32, 33, 34, 35, 36, 37, 38, 36, 39, 40, 1, 1, 1, 1, 1, 1, 1, 1], [41, 42, 43, 44, 45, 46, 47, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [48, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [49, 50, 51, 52, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [53, 54, 4, 55, 56, 57, 58, 59, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [60, 61, 62, 63, 64, 65, 66, 67, 68, 1, 1, 1, 1, 1, 1, 1, 1, 1], [69, 70, 71, 72, 73, 74, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [75, 76, 40, 77, 78, 79, 80, 81, 82, 1, 1, 1, 1, 1, 1, 1, 1, 1], [83, 84, 85, 57, 86, 87, 76, 88, 89, 90, 91, 92, 1, 1, 1, 1, 1, 1], [41, 93, 94, 95, 58, 96, 93, 97, 98, 99, 10, 100, 1, 1, 1, 1, 1, 1], [101, 102, 103, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [104, 105, 106, 107, 93, 108, 84, 10, 109, 110, 1, 1

#Création d'une Baseline

In [154]:
simple_model = tf.keras.models.Sequential()

simple_model.add(tf.keras.layers.SimpleRNN(15, input_shape=(1000,1000)))   # On définit une première version d'un modèle RNN simple

simple_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(0.01))                 

In [ ]:
simple_model.fit(padding,fit_reviews,epochs = 10)  # Il y a une incompatibilité entre les données et le paramètre input_shape

# Création d'un modèle plus complexe

In [ ]:
# model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.Embedding(input_dim= 5000, output_dim= 5000))   
# model.add(tf.keras.layers.Bidirectional(
#     tf.keras.layers.LSTM(5)
# ))
# model.add(tf.keras.layers.Dense(6, activation='relu'))
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [146]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim= 1000, output_dim= 1000))         # Les choix des paramètres seront expliqués dans le rapport
                                                                                # On compléxifie notre modèle
model.add(tf.keras.layers.LSTM(15))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(0.01))

In [148]:
model.fit(padding,fit_reviews,epochs = 100)

Epoch 1/100
4/4 [==============================] - 0s 29ms/step - loss: 0.1928
Epoch 2/100
4/4 [==============================] - 0s 26ms/step - loss: 0.1193
Epoch 3/100
4/4 [==============================] - 0s 28ms/step - loss: 0.0712
Epoch 4/100
4/4 [==============================] - 0s 27ms/step - loss: 0.0457
Epoch 5/100
4/4 [==============================] - 0s 27ms/step - loss: 0.0302
Epoch 6/100
4/4 [==============================] - 0s 29ms/step - loss: 0.0199
Epoch 7/100
4/4 [==============================] - 0s 28ms/step - loss: 0.0119
Epoch 8/100
4/4 [==============================] - 0s 27ms/step - loss: 0.0078
Epoch 9/100
4/4 [==============================] - 0s 33ms/step - loss: 0.0057
Epoch 10/100
4/4 [==============================] - 0s 28ms/step - loss: 0.0044
Epoch 11/100
4/4 [==============================] - 0s 27ms/step - loss: 0.0036
Epoch 12/100
4/4 [==============================] - 0s 28ms/step - loss: 0.0029
Epoch 13/100
4/4 [==============================]

In [149]:
model.save_weights("gdrive/MyDrive/Projet_NLP/model")

In [150]:
print(model.predict(np.expand_dims(padding[2] , axis = 0))) 

[[0.00065884]]


# Mise en place de la Pipeline

In [151]:
def analyse_sentiment(sentence):
  content = []
  tokenizer = nltk.RegexpTokenizer(r'\w+')
  english_stopwords = nltk.corpus.stopwords.words("english")

  stemmer = LancasterStemmer()
  phrase = tokenizer.tokenize(sentence.lower())
  for i in phrase:
    if i not in english_stopwords:
      content.append(stemmer.stem(i))
  #print(content) 


  maxlen = 0
  for word in content:
    if (len(word)) > maxlen:
      maxlen = len(word)
  #pint(maxlen) 

  from collections import defaultdict

  vocab = defaultdict(lambda: 0)
  vocab['<PAD>'] = 1

  for word in content:
    if word not in vocab:
      vocab[word] = len(vocab) + 1
  print(vocab)


  padding = []
  for word in content:
    if word in vocab:
      padding.append(vocab[word])                           # Si le mot appartient au vecteur de vocabulaire on ajoute l'indice correspondant sinon on ajoute un 0
    else:
      padding.append(0)

  print(padding)

  model.load_weights("gdrive/MyDrive/Projet_NLP/model")
  result = model.predict(np.expand_dims(padding ,axis = 0))
  print(result)

  if result <= 0.5:
    print("Your sentence is negative")                      # Il y a un problème dans la prédiction du modèle où la valeur reste inférieure à 0.5 bien que le modèle précédent soit fonctionnel
  else:
    print("Your sentence is positive")



In [152]:
analyse_sentiment("I love mondays")

defaultdict(<function analyse_sentiment.<locals>.<lambda> at 0x7f529c1b4d40>, {'<PAD>': 1, 'lov': 2, 'monday': 3})
[2, 3]
[[0.0911974]]
Your sentence is negative


# Appel à l'API yelp

In [ ]:
!pip install yelp
!pip install requests

In [82]:
import requests
import json
 
api_key='nCPX_SfMsQJne-Y6mmZsJNI2uYH53uIDavpQSIc1wbz2YsoCT5c5sWObDqyoZra9b2fWeeaKHv2bcBKdYGx6eXE4xyZUM18HUw2cSVaB4t4nzdZrYs1O5t9ZCvI4YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}

url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"   # On effectue la requête à l'API
req = requests.get(url, headers=headers)

In [83]:
parsed = json.loads(req.text)  # On récupére le contenu du fichier json
whole_data = parsed['reviews']
yelp_reviews = []
index = 0
for i in whole_data:
  yelp_reviews.append(whole_data[index]['text'])  # On sélectionne uniquement l'avis du consommateur
  index +=1

print(yelp_reviews)   # yelp_reviews contient bien les différentes reviews

["Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...", 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those crinkle cuts!) and a shake. Yum. Tbh,...', 'Before I moved to NY, my first ever trip to the city was in 2005 - this was when Shake Shack had this one lone original location, and it was an event to go...']


In [153]:
analyse_sentiment(yelp_reviews[1])

defaultdict(<function analyse_sentiment.<locals>.<lambda> at 0x7f52971fad40>, {'<PAD>': 1, 'hap': 2, 'city': 3, 'today': 4, 'far': 5, 'stop': 6, 'pick': 7, 'chees': 8, 'fri': 9, 'lov': 10, 'crinkl': 11, 'cut': 12, 'shak': 13, 'yum': 14, 'tbh': 15})
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[[0.07961369]]
Your sentence is negative
